In [1]:
## Imports
import BAC0
import pandas as pd

In [2]:
# Init variables
my_ip = '10.187.10.1/16'
bbmdIP = '10.187.113.206:47808'
bbmdTTL = 900

# Create Bacnet Connection
bacnet = BAC0.connect(ip=my_ip, bbmdAddress=bbmdIP, bbmdTTL=bbmdTTL)

2021-07-20 16:40:53,887 - INFO    | Starting BAC0 version 21.02.25 (Complete)
2021-07-20 16:40:53,891 - INFO    | Use BAC0.log_level to adjust verbosity of the app.
2021-07-20 16:40:53,892 - INFO    | Ex. BAC0.log_level('silence') or BAC0.log_level('error')
2021-07-20 16:40:53,893 - INFO    | Starting TaskManager
2021-07-20 16:40:53,895 - INFO    | Using ip : 10.187.10.1
2021-07-20 16:40:53,898 - INFO    | Starting app...
2021-07-20 16:40:53,900 - INFO    | BAC0 started
2021-07-20 16:40:53,901 - INFO    | Registered as Foreign Device
2021-07-20 16:40:53,905 - INFO    | Update Local COV Task started
2021-07-20 16:40:55,034 - INFO    | Server started : http://10.187.10.1:8111
 * Serving Flask app 'BAC0.web.FlaskServer' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.156:8111/ (Press CTRL+C to quit)


2021-07-20 16:40:55,248 - INFO    | 10.81.182.10 network number is 1
2021-07-20 16:40:57,284 - INFO    | 10.81.182.10 router to [1011]
2021-07-20 16:40:59,038 - INFO    | Addr : 10.81.182.10
2021-07-20 16:41:01,041 - INFO    | Found those networks : {1, 1011}
2021-07-20 16:41:01,044 - INFO    | Discovering network 1
2021-07-20 16:41:04,049 - INFO    | Discovering network 1011


In [4]:
devices = bacnet.devices
devices.head(5)

2021-07-20 16:42:20,111 - WARNING | APDU Abort Reject Reason : Timeout
2021-07-20 16:42:20,113 - WARNING | No response from controller Timeout


ValueError: not enough values to unpack (expected 2, got 1)

In [5]:
main_device = devices.iloc[0]
main_device

Manufacturer    Schneider Electric
Address               10.81.182.10
 Device ID                 3624521
Name: AS-P_3624521, dtype: object

In [6]:
controller = BAC0.device(main_device['Address'],main_device[' Device ID'],bacnet)

2021-07-20 12:11:45,554 - INFO    | Changing device state to DeviceDisconnected'>
2021-07-20 12:11:45,997 - INFO    | Changing device state to RPMDeviceConnected'>
2021-07-20 12:11:46,627 - INFO    | Device 3624521:[AS-P_3624521] found... building points list
2021-07-20 12:12:31,229 - INFO    | Ready!
2021-07-20 12:12:31,229 - INFO    | Device defined for normal polling with a delay of 10sec
2021-07-20 12:12:31,230 - INFO    | Polling started, values read every 10 seconds


In [8]:
controller.properties.objects_list

[('analogInput', 1),
 ('analogInput', 2),
 ('analogInput', 3),
 ('analogInput', 4),
 ('analogInput', 5),
 ('analogInput', 6),
 ('analogInput', 7),
 ('analogInput', 8),
 ('analogInput', 9),
 ('analogInput', 10),
 ('analogInput', 11),
 ('analogInput', 12),
 ('analogInput', 13),
 ('analogInput', 14),
 ('analogInput', 15),
 ('analogInput', 16),
 ('analogInput', 17),
 ('analogInput', 18),
 ('analogInput', 19),
 ('analogInput', 20),
 ('analogInput', 21),
 ('analogInput', 22),
 ('analogInput', 23),
 ('analogInput', 24),
 ('analogInput', 25),
 ('analogInput', 26),
 ('analogInput', 27),
 ('analogInput', 28),
 ('analogInput', 29),
 ('analogInput', 30),
 ('analogInput', 31),
 ('analogInput', 32),
 ('analogInput', 33),
 ('analogInput', 34),
 ('analogInput', 35),
 ('analogInput', 36),
 ('analogInput', 37),
 ('analogInput', 38),
 ('analogInput', 39),
 ('analogInput', 40),
 ('analogInput', 41),
 ('analogInput', 42),
 ('analogInput', 43),
 ('analogInput', 44),
 ('analogInput', 45),
 ('analogInput', 46

In [9]:
print('Points: %s | Properties: %s ' % (len(controller.points),len(controller.properties.objects_list)))

903

In [10]:
df = pd.DataFrame(controller.properties.objects_list, columns =['BACnet Type', 'Object ID'])

In [12]:
print(df['BACnet Type'].unique())
print(df[df['BACnet Type'] == 'file'])
print(df[df['BACnet Type'] == 'device'])
df = df[(df['BACnet Type']!='file') & (df['BACnet Type']!='device')]

['analogInput' 'analogOutput' 'binaryInput' 'binaryOutput' 'device' 'file'
 'multiStateInput']
    BACnet Type  Object ID
816        file          1
    BACnet Type  Object ID
815      device    3624521


In [13]:
for index, row in df.iterrows():
    try:
        var_type=row['BACnet Type']
        _id=row['Object ID']
        val = bacnet.read('%s %s %s %s'%(main_device['Address'],var_type,_id,'presentValue'))
        print('%s (%s) -> %s'%(var_type,_id,val))
    except Exception as e:
        print(e)

analogInput (1) -> 25.5
analogInput (2) -> 23.500001907348633
analogInput (3) -> 40.0
analogInput (4) -> 27.5
analogInput (5) -> 25.5
analogInput (6) -> 23.500001907348633
analogInput (7) -> -89.9800033569336


KeyboardInterrupt: 

In [14]:
csv_df = pd.read_csv('../input/primer_maig.csv')
csv_df

,BACnet Name,BACnet Type,Object ID,Description
0,CL1\FiltsAlarma,binary-input,"<binary-input, 41>","0=OK, 1=Alarma"
1,CL1\HorariActiu,binary-input,"<binary-input, 1>","0=Inactiu, 1=Actiu"
2,CL1\ImpTª,analog-input,"<analog-input, 37>",ºC
3,CL1\LimitCalor,analog-output,"<analog-output, 5>",ºC
4,CL1\LimitFred,analog-output,"<analog-output, 6>",ºC
...,...,...,...,...
895,Consum General GAS Zigbee,analog-input,"<analog-input, 249>",Consum de GAS en m3 Zigbee
896,HR Ambient PB Zigbee,analog-input,"<analog-input, 250>",HR Ambient PB Zigbee
897,HR Exterior Zigbee,analog-input,"<analog-input, 251>",HR Exterior Zigbee
898,Tª Ambient PB Zigbee,analog-input,"<analog-input, 252>",Tª Ambient PB Zigbee


In [15]:
def take_id(str):
    return int(str.strip().split(',')[1][:-1])

def camelCase(st):
    output = ''.join(x for x in st.title() if x.isalnum())
    return output[0].lower() + output[1:]

In [16]:
csv_df['Object ID'] = csv_df['Object ID'].map(lambda i: take_id(i))

In [17]:
csv_df['BACnet Type'] = csv_df['BACnet Type'].map(lambda i: camelCase(i.replace('-',' ')))

In [18]:
csv_df

,BACnet Name,BACnet Type,Object ID,Description
0,CL1\FiltsAlarma,binaryInput,41,"0=OK, 1=Alarma"
1,CL1\HorariActiu,binaryInput,1,"0=Inactiu, 1=Actiu"
2,CL1\ImpTª,analogInput,37,ºC
3,CL1\LimitCalor,analogOutput,5,ºC
4,CL1\LimitFred,analogOutput,6,ºC
...,...,...,...,...
895,Consum General GAS Zigbee,analogInput,249,Consum de GAS en m3 Zigbee
896,HR Ambient PB Zigbee,analogInput,250,HR Ambient PB Zigbee
897,HR Exterior Zigbee,analogInput,251,HR Exterior Zigbee
898,Tª Ambient PB Zigbee,analogInput,252,Tª Ambient PB Zigbee


In [19]:
x = pd.merge(df,csv_df,on=['BACnet Type','Object ID'])

In [20]:
x

,BACnet Type,Object ID,BACnet Name,Description
0,analogInput,1,FC1\Occupied Cool Setpoint,Consigna Real Ocupació Fred
1,analogInput,2,FC1\Occupied Heat Setpoint,Consigna Real Ocupació Calor
2,analogInput,3,FC1\Room Humidity,Humitat de Sala
3,analogInput,4,FC1\Room Temperature,Temperatura de Sala
4,analogInput,5,FC2\Occupied Cool Setpoint,Consigna Real Ocupació Fred
...,...,...,...,...
895,multiStateInput,82,FC41\System Mode,"Mode de funcionament 1:Off, 2:Auto, 3:Fred, 4:..."
896,multiStateInput,83,FC42\EffectOcc,Estat D'Ocupació
897,multiStateInput,84,FC42\System Mode,"Mode de funcionament 1:Off, 2:Auto, 3:Fred, 4:..."
898,multiStateInput,85,FC43\EffectOcc,Estat D'Ocupació


In [21]:
aux = csv_df[['BACnet Type','Object ID']]

In [22]:
pd.concat([df,aux]).drop_duplicates(keep=False)

,BACnet Type,Object ID
572,binaryInput,46


In [24]:
# https://bac0.readthedocs.io/en/latest/histories.html
controller['FC1\Occupied Cool Setpoint'].history

2021-07-20 12:12:13.159994    25.5
2021-07-20 12:12:47.835723    25.5
Name: AS-P_3624521/FC1\Occupied Cool Setpoint, dtype: float64